# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function
    
    G = nx.DiGraph()
    demand_dict = dict()  # save demand for each node
    network_file = open(infile)  # open the data
    # read the data to the network
    for line in network_file:
        the_line = line.strip().split()
        if len(the_line) == 0:
            continue
        if the_line[0] == 'p':  # p min <nodes> <arcs>
            node_number = int(the_line[2])
        elif the_line[0] == 'a':  # a <v> <w> <low> <cap> <cost>
            if (the_line[1], the_line[2]) in G.edges():
                # reduction to standard digraph
                node_number += 1
                G.add_node(node_number)  # add dummy node
                G.add_edge(the_line[1], node_number, {'weight': float(the_line[5]) / 2, 
                                                           'capacity': int(the_line[4])})
                G.add_edge(node_number, the_line[2], {'weight': float(the_line[5]) / 2, 
                                                           'capacity': int(the_line[4])})
            else:
                G.add_edge(the_line[1], the_line[2], {'weight': float(the_line[5]), 'capacity': int(the_line[4])})
        elif the_line[0] == 'n':  # n <id> <flow>
            demand_dict[the_line[1]] = int(the_line[2])
            
    # add demand to each node
    for every_node in nx.nodes(G):
        if every_node in demand_dict.keys():
            G.node[every_node]['demand'] = demand_dict[every_node]
        else:
            G.node[every_node]['demand'] = 0

    return G

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    # construct the lp problem
    min_cost_lp = pulp.LpProblem("min cost lp", pulp.LpMinimize)
    
    # add variables, and construct obj function
    flow = dict()
    weighted_flow = dict()
    for every_edge in G.edges():
        upper_bound = G.edge[every_edge[0]][every_edge[1]]['capacity']
        name = "f_" + str(every_edge[0]) + "_" + str(every_edge[1])
        flow[every_edge] = pulp.LpVariable(name, lowBound=0, upBound=upper_bound, cat='Continuous')
    # obj function
    min_cost_lp += pulp.lpSum(G.edge[every_edge[0]][every_edge[1]]['weight'] * flow[every_edge] \
                             for every_edge in G.edges())

    # constraints
    for every_node, node_demand in G.nodes(data = True):
        if G.predecessors(every_node) == []:
            min_cost_lp += pulp.lpSum(flow[(every_node, next_node)] \
                                        for next_node in G.neighbors(every_node)) == -node_demand
        elif G.neighbors(every_node) == []:
            min_cost_lp += pulp.lpSum([flow[(pre_node, every_node)] \
                                       for pre_node in G.predecessors(every_node)]) == node_demand
        else:
            min_cost_lp += pulp.lpSum([flow[(pre_node, every_node)] for pre_node in G.predecessors(every_node)]) \
                              - pulp.lpSum(flow[(every_node, next_node)] for next_node in G.neighbors(every_node)) \
                              == node_demand
    # solve lp
    min_cost_lp.solve()

    return pulp.value(min_cost_lp.objective)

The following will check that the LP finds the same optimal values as previously.

In [4]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
